# Common imports

In [ ]:
#%matplotlib inline
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import copy
import time

# Provided functions
### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map):
    fig = plt.figure()
    plt.imshow(albedo_image, cmap='gray')
    plt.axis('off')
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 20)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [ ]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure()
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0])
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1])
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2])

# Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    
    ambimage_cpy = copy.deepcopy(ambimage)
    #imarray_cpy = copy.deepcopy(imarray)
    imarray_cpy = imarray
    processed_imarray = np.clip(imarray_cpy - ambimage_cpy[..., np.newaxis], 0, 255)
    processed_imarray = processed_imarray/255
    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    #light_dirs_cpy = copy.deepcopy(light_dirs)
    #imarray_cpy = copy.deepcopy(imarray)
    light_dirs_cpy = light_dirs
    imarray_cpy = imarray
    size_arr = imarray_cpy.shape    
    imarray_cpy = imarray_cpy.T.reshape(size_arr[2],size_arr[1]*size_arr[0])    
    gxy = np.linalg.lstsq(light_dirs_cpy,imarray_cpy,rcond=-1)[0]
    #print("LSS range", np.amax(gxy), np.amin(gxy))
    #print("gxy shape", gxy.shape)
    albxy = np.sqrt(np.sum(gxy**2, axis = 0))
    albedo_image = albxy.reshape((size_arr[0], size_arr[1]), order = 'F')
    surnmlxy = gxy/albxy
    surface_normals = surnmlxy.T.reshape((size_arr[0], size_arr[1], 3), order = 'F')
    
    return albedo_image, surface_normals
    

In [ ]:
def random_path(fx_cum, fy_cum, ntimes = 2):
    
    height_map = np.full((fx_cum.shape[0],fx_cum.shape[1]), fx_cum[0,0], dtype = np.float64)
    
    for num_paths in range(ntimes):
        
        for i in range(fx_cum.shape[0]):
            for j in range(fx_cum.shape[1]):
                prev = [0,0]
                while (prev != [i,j]):                
                    if (prev[1] == j):
                        rdir = 0
                    elif (prev[0] == i):
                        rdir = 1
                    else:
                        rdir = np.random.randint(2)

                    if rdir == 0:
                        rlen = np.random.randint(prev[0]+1, i+1)
                        height_map[i,j] += (fx_cum[rlen,prev[1]] - fx_cum[prev[0],prev[1]])
                        prev[0] = rlen
                    else:
                        rlen = np.random.randint(prev[1]+1, j+1)
                        height_map[i,j] += (fy_cum[prev[0],rlen] - fy_cum[prev[0],prev[1]])
                        prev[1] = rlen
    
                    
                    
    return height_map/ntimes                
                
            
             
            
    
    

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
        Convention used " h is x and w is y"
    """
    #In image array, x is along height and y is along width 
    fy = surface_normals[:,:,0]/surface_normals[:,:,2]
    fx = surface_normals[:,:,1]/surface_normals[:,:,2]
    
    fy_cum = np.cumsum(fy, axis = 1)
    fx_cum = np.cumsum(fx, axis = 0)
    
    height_map1 =  fx_cum + fy_cum[0:1, :]
    height_map2 =  fy_cum + fx_cum[:, 0:1]
   
    
    #print("pd dtype", fy_cum.dtype, fx_cum.dtype)
    #print("pd range", np.amin(fx_cum), np.amax(fy_cum))
    
    if integration_method == 'row':
        return height_map1
    elif integration_method == 'column':
        return height_map2
    elif integration_method == 'average':
        return (height_map2 + height_map1)/2
    elif integration_method == 'random':
        return random_path(fx_cum, fy_cum, ntimes = 60)
    else:
        return heightmap1
    
    
    
    
    

# Main function

In [ ]:
root_path = '../croppedyale/'
subject_name = 'yaleB01'
integration_method = 'random'
save_flag = True

full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,64)

'''print("input sizes: ambient, image array, light dirs", ambient_image.shape,imarray.shape,light_dirs.shape)
print("ambient range", np.amax(ambient_image),np.amin(ambient_image))'''

processed_imarray = preprocess(ambient_image, imarray)
'''print("processed size", processed_imarray.shape)
print("processed range", np.amax(processed_imarray), np.amin(processed_imarray))'''

albedo_image, surface_normals = photometric_stereo(processed_imarray,light_dirs)

'''print("albedo + surface normal shape ", albedo_image.shape,surface_normals.shape )

print("Albedo range", np.amax(albedo_image), np.amin(albedo_image))
print("surface normal range" , np.amax(surface_normals), np.amin(surface_normals))

#Check for unit normal
mag = np.sqrt(surface_normals[:,:,0]**2 + surface_normals[:,:,1] **2 + surface_normals[:,:,2]**2) 
print("UN", mag.shape, np.amax(mag), np.amin(mag))'''

height_map = get_surface(surface_normals, integration_method)


#print("height_map range", np.amin(height_map), np.amax(height_map))

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
total_time = 0
ntimes = 20
for i in range(ntimes):
    start = time.process_time()
    height_map = get_surface(surface_normals, integration_method)
    total_time  = total_time + (time.process_time() - start)

print("Average time for " + integration_method + " is ",   total_time/ntimes)


In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
%matplotlib notebook

In [ ]:
display_output(albedo_image, height_map)